<a href="https://colab.research.google.com/github/Adetoki-timilehin/Adetoki-timilehin/blob/main/Brain_Tumor_Detection_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


In [ ]:
print("TF version:", tf.__version__)

TF version: 2.5.0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from sklearn.utils import shuffle
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from sklearn.metrics import confusion_matrix, classification_report
import ipywidgets as widgets
import io
from PIL import Image
from IPython.display import display, clear_output
from warnings import filterwarnings

DATA PREPARATION

In [ ]:
labels = ["glioma_tumor", "meningioma_tumor", "pituitary_tumor", "no_tumor"]

In [ ]:
x = []
y = []

img_size = 150
for i in labels:
  folder_path = os.path.join("drive/MyDrive/BrainTumorClassificationMri/Training", i)
  for j in tqdm(os.listdir(folder_path)):
    img = cv2.imread(os.path.join(folder_path, j))
    img = cv2.resize(img, (img_size, img_size))
    x.append(img)
    y.append(i)


for i in labels:
  folder_path = os.path.join("drive/MyDrive/BrainTumorClassificationMri", "Testing", i)
  for j in tqdm(os.listdir(folder_path)):
    img = cv2.imread(os.path.join(folder_path, j))
    img = cv2.resize(img, (img_size, img_size))
    x.append(img)
    y.append(i)

x = np.array(x)
y = np.array(y)

100%|██████████| 105/105 [00:37<00:00,  2.82it/s]


In [ ]:
datagen = ImageDataGenerator(rotation_range = 30,
                              width_shift_range = 0.1,
                              height_shift_range = 0.1,
                              zoom_range = 0.2,
                              horizontal_flip = True)

datagen.fit(x)
x.shape

(3264, 150, 150, 3)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 42)

In [ ]:
y_train_int = []
for i in y_train:
  y_train_int.append(labels.index(i))
y_train = y_train_int
y_train = tf.keras.utils.to_categorical(y_train)

y_test_int = []
for i in y_test:
  y_test_int.append(labels.index(i))
y_test = y_test_int
y_test = tf.keras.utils.to_categorical(y_test)

TRANSFER LEARNING

In [ ]:
effnet = EfficientNetB0(weights = 'imagenet', include_top = False, input_shape = (img_size, img_size, 3))

16711680/16705208 [==============================] - 0s 0us/step


In [ ]:
model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate = 0.5)(model)
model = tf.keras.layers.Dense(4, activation = 'softmax')(model)
model = tf.keras.models.Model(inputs = effnet.input, outputs = model)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 150, 150, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [ ]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5", monitor = 'val_accuracy', save_best_only = True, mode = 'auto', verbose = 1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001, mode = 'auto', verbose = 1)

In [ ]:
brain = model.fit(x_train, y_train, validation_split = 0.1, epochs = 12, verbose = 1, batch_size = 32, callbacks = [tensorboard, checkpoint, reduce_lr])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/12
83/83 [==============================] - 67s 342ms/step - loss: 0.4497 - accuracy: 0.8384 - val_loss: 0.8735 - val_accuracy: 0.7551

Epoch 00001: val_accuracy improved from -inf to 0.75510, saving model to effnet.h5
Epoch 2/12
83/83 [==============================] - 24s 293ms/step - loss: 0.1934 - accuracy: 0.9315 - val_loss: 1.0153 - val_accuracy: 0.7483

Epoch 00002: val_accuracy did not improve from 0.75510
Epoch 3/12
83/83 [==============================] - 25s 296ms/step - loss: 0.1122 - accuracy: 0.9599 - val_loss: 0.4884 - val_accuracy: 0.8912

Epoch 00003: val_accuracy improved from 0.75510 to 0.89116, saving model to effnet.h5
Epoch 4/12
83/83 [==============================] - 24s 295ms/step - loss: 0.1057 - accuracy: 0.9678 - val_loss: 1.0648 - val_accuracy: 0.7687

Epoch 00004: val_accuracy did not improve from 0.89116
Epoch 5/12
83/83 [==============================] - 24s 293ms/step - loss: 0.0923 - accuracy: 0.9694 - val_loss: 0.3234 - val_accuracy: 0.9286

E

MAKING PREDICTIONS

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred, axis = 1)
y_test_index = np.argmax(y_test, axis = 1)

In [ ]:
print(classification_report(y_test_index, pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       117
           1       0.96      0.89      0.93        84
           2       0.99      1.00      0.99        85
           3       0.95      1.00      0.98        41

    accuracy                           0.96       327
   macro avg       0.96      0.97      0.96       327
weighted avg       0.96      0.96      0.96       327

